# L4: Optimize DSPy Agent with DSPy Optimizer

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

import os

os.environ["OPENAI_API_KEY"] = get_openai_api_key()

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.</p>

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

In [2]:
import mlflow

In [3]:
from helper import get_mlflow_tracking_uri

mlflow_tracking_uri = get_mlflow_tracking_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)

In [4]:
mlflow.set_experiment("dspy_course_4")

2025/06/07 14:54:35 INFO mlflow.tracking.fluent: Experiment with name 'dspy_course_4' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/329069445684618311', creation_time=1749308075110, experiment_id='329069445684618311', last_update_time=1749308075110, lifecycle_stage='active', name='dspy_course_4', tags={}>

In [5]:
mlflow.dspy.autolog(log_evals=True, log_compiles=True, log_traces_from_compile=True)

In [6]:
import dspy

dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

## Build a RAG Agent

In [7]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

In [8]:
import json

# Load trainset
trainset = []
with open("trainset.jsonl", "r") as f:
    for line in f:
        trainset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

# Load valset
valset = []
with open("valset.jsonl", "r") as f:
    for line in f:
        valset.append(dspy.Example(**json.loads(line)).with_inputs("question"))

In [9]:
# Overview of the dataset.
print(trainset[0])

Example({'question': 'Are Smyrnium and Nymania both types of plant?', 'answer': 'yes'}) (input_keys={'question'})


In [10]:
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,
    auto="light",
    num_threads=16
)

In [11]:
dspy.cache.load_memory_cache("./memory_cache.pkl")

In [12]:
optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,
)

2025/06/07 14:54:36 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '207f8d710df64560962a56fd1cd04cf3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current dspy workflow
2025/06/07 14:54:36 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 20
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/06/07 14:54:36 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/06/07 14:54:36 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/06/07 14:54:36 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 18%|█▊        | 18/100 [00:02<00:11,  7.33it/s]


Bootstrapped 4 full traces after 18 examples for up to 1 rounds, amounting to 18 attempts.


Bootstrapping set 4/6


...

Average Metric: 54.00 / 100 (54.0%): 100%|██████████| 100/100 [00:24<00:00,  4.06it/s]

2025/06/07 14:58:16 INFO dspy.evaluate.evaluate: Average Metric: 54 / 100 (54.0%)
2025/06/07 14:58:16 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 54.0
2025/06/07 14:58:16 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [31.0, 50.0, 49.0, 49.0, 54.0]
2025/06/07 14:58:16 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 54.0
2025/06/07 14:58:16 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/06/07 14:58:16 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/06/07 14:58:16 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 54.0!



🏃 View run eval_full_4 at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311/runs/86e16d1834ae4105878f21b962a080d5
🧪 View experiment at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311


🏃 View run serious-midge-732 at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311/runs/207f8d710df64560962a56fd1cd04cf3
🧪 View experiment at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311


[Trace(request_id=6d67a44c278c4355b08a692ae64f0608), Trace(request_id=709e3cf3db61466f8775e8bbf016ce80), Trace(request_id=6709d2a5ef304939afe95df8923cdb10), Trace(request_id=9dfb897a746543eebf90d84e1182e6ad), Trace(request_id=565983987f1445cdac2bd2a3645b76ba), Trace(request_id=0fe47b1bd6b24c7a8f73adcdbbb81384), Trace(request_id=39af8595d32949398e4b2d21d5ae09b6), Trace(request_id=71308efdc2ae422391aea2da8c3f969a), Trace(request_id=5042089c2c5e433998b7c59c7150e025), Trace(request_id=2726d0a33bde4c738845dfa828edc54a)]

In [13]:
optimized_react.react.signature

StringSignature(question, trajectory -> next_thought, next_tool_name, next_tool_args
    instructions="Given the fields `question`, produce the fields `answer`.\n\nYou are an Agent. In each episode, you will be given the fields `question` as input. And you can see your past trajectory so far.\nYour goal is to use one or more of the supplied tools to collect any necessary information for producing `answer`.\n\nTo do this, you will interleave next_thought, next_tool_name, and next_tool_args in each turn, and also when finishing the task.\nAfter each tool call, you receive a resulting observation, which gets appended to your trajectory.\n\nWhen writing next_thought, you may reason about the current situation and plan for future steps.\nWhen selecting the next_tool_name and its next_tool_args, the tool must be one of:\n\n(1) search_wikipedia. It takes arguments {'query': {'type': 'string'}} in JSON format.\n(2) finish, whose description is <desc>Marks the task as complete. That is, signals

In [14]:
optimized_react.react.demos

[Example({'augmented': True, 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?', 'trajectory': '[[ ## thought_0 ## ]]\nI need to find out which studio produced both "That Darn Cat!" and "Never a Dull Moment." This information is likely available on Wikipedia, so I will search for it there.\n\n[[ ## tool_name_0 ## ]]\nsearch_wikipedia\n\n[[ ## tool_args_0 ## ]]\n{"query": "That Darn Cat! and Never a Dull Moment studio production"}\n\n[[ ## observation_0 ## ]]\n[1] «That Darn Cat! | That Darn Cat! is a 1965 American Walt Disney Productions thriller comedy film starring Hayley Mills (in her last of the six films she made for the Walt Disney Studios) and Dean Jones (starring in his first film for Disney) in a story about bank robbers, a kidnapping and a mischievous cat. The film was based on the 1963 novel "Undercover Cat" by Gordon and Mildred Gordon and was directed by Robert Stevenson. The title song was written by the Sherman Brothers and sung by Bo

In [15]:
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,
    devset=valset,
    display_table=True,
    display_progress=True,
    num_threads=24,
)

In [16]:
original_score = evaluator(react)
print(f"Original score: {original_score}")

Average Metric: 31.00 / 100 (31.0%): 100%|██████████| 100/100 [00:15<00:00,  6.33it/s]

2025/06/07 14:58:33 INFO dspy.evaluate.evaluate: Average Metric: 31 / 100 (31.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...","Steve McQueen, known as ""the king of cool,"" starred in the movie ""...","The movie is ""The Great Escape.""",
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,"{'thought_0': 'I need to determine which individual, Robert Kardas...",Robert Kardashian's family is well-known for their reality TV show...,Robert Kardashian's family had their own reality TV show.,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in the film ""Shadows ...","I searched for information about the cast of the 1986 film ""Shadow...",There is no information available about a Russian ballerina in the...,
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...",Nehemiah appointed Amashsai to work at the temple in Jerusalem. Th...,"The meaning of the name of the man who appointed Amashsai, Nehemia...",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements or ...,To gain access to 173 countries and territories with an Austrian p...,"In addition to the Austrian passport, travelers may need to obtain...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the name of the American actress...,The American actress and singer-songwriter known for her role as P...,2007,
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The animated creatures that are the title characters of the film b...,The animated creatures that are the title characters of the film b...,
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,{'thought_0': 'I need to verify the nationalities and contribution...,Both Dorothy Arzner and Richard Wallace were confirmed to be Ameri...,"No, neither Dorothy Arzner nor Richard Wallace were French film di...",


🏃 View run salty-midge-199 at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311/runs/f2af7a569f824fc4b8727348e53622dc
🧪 View experiment at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311
Original score: 31.0


[Trace(request_id=83172fe3ca3b42778512b5818432317d), Trace(request_id=3db2aa08aa154fb69ea47e89346d86c4), Trace(request_id=3150ccac509940b1aed29bcdd73daeb3), Trace(request_id=9eb4df32d4dd4656b8f0424107fa55c5), Trace(request_id=0f98096f3fc443fb82b7faff90e38eec), Trace(request_id=13c275a07995470594427f5e1e87f84a), Trace(request_id=824d8ef7563c49cfb10f260d49f15f86), Trace(request_id=0f620723eb7b4b2196a390e9d582f56e), Trace(request_id=a40d70af10e24a61bb125206e4c7eb00), Trace(request_id=5df22a7b8fa8426abc793cc73e759ba6)]

In [17]:
optimized_score = evaluator(optimized_react)
print(f"Optimized score: {optimized_score}")

Average Metric: 54.00 / 100 (54.0%): 100%|██████████| 100/100 [00:19<00:00,  5.02it/s]

2025/06/07 14:58:53 INFO dspy.evaluate.evaluate: Average Metric: 54 / 100 (54.0%)


,question,example_answer,trajectory,reasoning,pred_answer,answer_exact_match
0,"What movie did ""the king of cool"" play in with Bud Ekins as his st...","""The Great Escape""","{'thought_0': 'I need to find out which movie ""the king of cool"" s...",I found that Bud Ekins was Steve McQueen's stunt double in the fil...,The Great Escape,✔️ [True]
1,whos family had their own reality tv show. Robert Kardashian or Ma...,their family reality television series,{'thought_0': 'I need to find out which family had their own reali...,"The Kardashian family, associated with Robert Kardashian, has thei...",Robert Kardashian,
2,Which star in Shadows in Paradise is a Russian ballerina?,Sofya Skya,"{'thought_0': 'I need to find out which star in ""Shadows in Paradi...","In my search for the cast of ""Shadows in Paradise,"" I found that t...",Sofya Skya,✔️ [True]
3,What was the meaning of the name of the man who appointed Amashsai?,comforter,"{'thought_0': ""I need to find out who appointed Amashsai and the m...","Amashsai was appointed by Nehemiah, and the name Amasai, which is ...","""Burdensome""",
4,"In addition to the Austrian passport, what is needed to gain acces...",national identity card,{'thought_0': 'I need to find out what additional requirements are...,The search results indicate that Austrian citizens have visa-free ...,"A valid Austrian passport, and potentially a visa or health docume...",
...,...,...,...,...,...,...
95,"What date did the American actress and singer-songwriter, known fo...","April 19, 1994",{'thought_0': 'I need to find out the release date of the first al...,I found that the American actress and singer-songwriter Katey Saga...,"April 19, 1994",✔️ [True]
96,What animated creatures were the title characters of the film whic...,seals,{'thought_0': 'I need to identify the animated creatures that were...,The question pertains to animated creatures that are the title cha...,"Fairies (specifically Puck, Titania, and Oberon)",
97,The 1925 Saint Mary's Gaels football team represented what private...,Saint Mary's College of California,"{'thought_0': ""I need to find out which private, coeducational col...",The 1925 Saint Mary's Gaels football team represented Saint Mary's...,Saint Mary's College of California,✔️ [True]
98,Were Dorothy Arzner and Richard Wallace both French film directors?,no,"{'thought_0': ""I need to determine if both Dorothy Arzner and Rich...","I found that Dorothy Arzner was an American film director, and Ric...","No, neither Dorothy Arzner nor Richard Wallace were French film di...",


🏃 View run charming-loon-825 at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311/runs/741de0f6a4f3415ca2fedd3306272c7a
🧪 View experiment at: https://s172-29-69-253p8080.lab-aws-production.deeplearning.ai/#/experiments/329069445684618311
Optimized score: 54.0


[Trace(request_id=cc61517b97f64fd6a21368ba657e6411), Trace(request_id=faf601a1152e4bd2844800599b82de3e), Trace(request_id=d9050eae71314759b4be0153a77b8699), Trace(request_id=82ed693c456b412895f9f5ae93e2a951), Trace(request_id=0a26e105ea6345a3a5b2aeb93ec93d6e), Trace(request_id=537a0886ab654acf897dd92f82c8cd0b), Trace(request_id=4a96322b9dc14fc28b807620e97105d3), Trace(request_id=8f016a8969a24eb798e3e6f91b363395), Trace(request_id=d7816885c3ab4a3e8e46bd3d872c8839), Trace(request_id=c5812c2e9623400f9ca1cef82d83ca44)]